In [2]:
from __future__ import print_function  

In [3]:
import numpy as np
import json
import os 

from keras.preprocessing import sequence


In [4]:
sequences_file = os.path.join('..', 'data', 'protein-seqs-2018-01-16-131956.txt')
functions_file = os.path.join('..', 'data', 'protein-functions-2018-01-16-131956.txt')

In [5]:
with open(functions_file) as fn_file:
    has_function = json.load(fn_file)

In [6]:
has_function  # just to see what we have loaded 

['P27361',
 'P53779',
 'Q9UHC1',
 'Q9NYL2',
 'O15440',
 'P33527',
 'Q92887',
 'O15438',
 'O15439',
 'Q5T3U5',
 'P42345',
 'O75648',
 'Q16659',
 'Q8NB16',
 'Q02750',
 'O95255',
 'O95396',
 'O43196',
 'P46734',
 'P49914',
 'Q6DT37',
 'Q9H3H1',
 'Q9HCE1',
 'P52564',
 'Q9Y5S2',
 'Q96J65',
 'P20585',
 'O15457',
 'P52701',
 'Q5VT25',
 'A7E2Y1',
 'Q9UKN7',
 'Q9Y6X6',
 'Q96MN2',
 'P13535',
 'Q32MK0',
 'O43795',
 'O00159',
 'Q9UM54',
 'Q13402',
 'Q9UKX2',
 'P13533',
 'Q9H1R3',
 'Q8WXR4',
 'Q6PIF6',
 'B2RTY4',
 'Q6IA69',
 'Q86W25',
 'Q9UKX3',
 'Q7Z406',
 'P12882',
 'Q15746',
 'O15146',
 'Q9Y623',
 'P12883',
 'P35579',
 'B0I1T2',
 'Q9Y4I1',
 'Q96JP2',
 'Q8IUG5',
 'Q9UBC5',
 'O00160',
 'P53602',
 'Q92614',
 'P35749',
 'P11055',
 'Q9Y2K3',
 'Q86YV6',
 'Q9HD67',
 'Q8N1T3',
 'Q9ULV0',
 'Q13459',
 'Q86W26',
 'P59046',
 'Q9UJ70',
 'Q86W24',
 'Q13232',
 'Q9H0A0',
 'P22392',
 'Q8IY84',
 'Q86WI3',
 'Q96P20',
 'Q86UW6',
 'Q8IVL1',
 'O75414',
 'Q8NG66',
 'Q96PY6',
 'P35580',
 'Q96H55',
 'O94832',
 'Q12965',

In [7]:
max_sequence_size = 500   # any sequence longer than this, we ignore (just for now) 

In [8]:
X = []    #datapoints       # sequences in the same order corresponding to elements of p 
y = []    #labels       # output class: 1 if protein has the function, 0 if not 

In [9]:
# for seeing how many examples we've found for each class 
pos_examples = 0
neg_examples = 0   

In [10]:
with open(sequences_file) as f:
    for line in f:
        ln = line.split(',')
        protein_id = ln[0].strip() #first one is protein_id 
        seq = ln[1].strip()        #second one is seq

        # we're doing this to reduce input size
        if len(seq) >= max_sequence_size:
            continue
        
        print(line)
        
        X.append(seq)
        
        if protein_id in has_function: 
            y.append(1) 
            pos_examples += 1 
        else: 
            y.append(0) 
            neg_examples += 1 

P27361,MAAAAAQGGGGGEPRRTEGVGPGVPGEVEMVKGQPFDVGPRYTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLREIQILLRFRHENVIGIRDILRASTLEAMRDVYIVQDLMETDLYKLLKSQQLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLINTTCDLKICDFGLARIADPEHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINMKARNYLQSLPSKTKVAWAKLFPKSDSKALDLLDRMLTFNPNKRITVEEALAHPYLEQYYDPTDEPVAEEPFTFAMELDDLPKERLKELIFQETARFQPGVLEAP

P53779,MSLHFLYYCSEPTLDVKIAFCQGFDKQVDVSYIAKHYNMSKSKVDNQFYSVEVGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDAVLDRNVAIKKLSRPFQNQTHAKRAYRELVLMKCVNHKNIISLLNVFTPQKTLEEFQDVYLVMELMDANLCQVIQMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTAGTSFMMTPYVVTRYYRAPEVILGMGYKENVDIWSVGCIMGEMVRHKILFPGRDYIDQWNKVIEQLGTPCPEFMKKLQPTVRNYVENRPKYAGLTFPKLFPDSLFPADSEHNKLKASQARDLLSKMLVIDPAKRISVDDALQHPYINVWYDPAEVEAPPPQIYDKQLDEREHTIEEWKELIYKEVMNSEEKTKNGVVKGQPSPSGAAVNSSESLPPSSSVNDISSMSTDQTLASDTDSSLEASAGPLGCCR

Q15049,MTQEPFREELAYDRMPTLERGRQDPASYAPDAKPSDLQLSKRLPPCFSHKTWVFSVLMGSCLLVTSGFSLYLGNVFPAEMDYLRCAAGSCIPSAIVSFTVSRRNANVIPNFQILFVSTFAVTTTCLIWFGCK

In [11]:
print("Positive Examples: %d" % pos_examples)
print("Negative Examples: %d" % neg_examples)  # Total is different because we ignored longer sequences 

Positive Examples: 2
Negative Examples: 5


In [12]:
def sequence_to_indices(sequence):
    """Convert amino acid letters to indices. 
       _ means no amino acid (used for padding to accommodate for variable length)"""
    
    try:
        acid_letters = ['_', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
                'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

        indices = [acid_letters.index(c) for c in list(sequence)]
        '''
        we will loop over the sequence and return index of letter according to the acid_letter
        e.g.,
        
        [1,2] = sequence_to_indices("AC")
        
        '''
        return indices
    except Exception:
        print(sequence)
        raise Exception

In [13]:
sequence_to_indices('AD')  # just testing 

[1, 3]

In [15]:
X_all = [] 
for i in range(len(X)): 
    x = sequence_to_indices(X[i]) # convert from sequence into indices
    X_all.append(x)               #save in X_all
    

In [16]:
X_all = np.array(X_all)          #convert into numpy array for fast modification and speed
y_all = np.array(y)

In [19]:
print(y[0])                     #label = 1 (mnx exhbit apt-binding)
print(X_all[0])                 
print(len(X_all[0]))

1
[11, 1, 1, 1, 1, 1, 14, 6, 6, 6, 6, 6, 4, 13, 15, 15, 17, 4, 6, 19, 6, 13, 6, 19, 13, 6, 4, 19, 4, 11, 19, 9, 6, 14, 13, 5, 3, 19, 6, 13, 15, 22, 17, 14, 10, 14, 22, 8, 6, 4, 6, 1, 22, 6, 11, 19, 16, 16, 1, 22, 3, 7, 19, 15, 9, 17, 15, 19, 1, 8, 9, 9, 8, 16, 13, 5, 4, 7, 14, 17, 22, 2, 14, 15, 17, 10, 15, 4, 8, 14, 8, 10, 10, 15, 5, 15, 7, 4, 12, 19, 8, 6, 8, 15, 3, 8, 10, 15, 1, 16, 17, 10, 4, 1, 11, 15, 3, 19, 22, 8, 19, 14, 3, 10, 11, 4, 17, 3, 10, 22, 9, 10, 10, 9, 16, 14, 14, 10, 16, 12, 3, 7, 8, 2, 22, 5, 10, 22, 14, 8, 10, 15, 6, 10, 9, 22, 8, 7, 16, 1, 12, 19, 10, 7, 15, 3, 10, 9, 13, 16, 12, 10, 10, 8, 12, 17, 17, 2, 3, 10, 9, 8, 2, 3, 5, 6, 10, 1, 15, 8, 1, 3, 13, 4, 7, 3, 7, 17, 6, 5, 10, 17, 4, 22, 19, 1, 17, 15, 20, 22, 15, 1, 13, 4, 8, 11, 10, 12, 16, 9, 6, 22, 17, 9, 16, 8, 3, 8, 20, 16, 19, 6, 2, 8, 10, 1, 4, 11, 10, 16, 12, 15, 13, 8, 5, 13, 6, 9, 7, 22, 10, 3, 14, 10, 12, 7, 8, 10, 6, 8, 10, 6, 16, 13, 16, 14, 4, 3, 10, 12, 2, 8, 8, 12, 11, 9, 1, 15, 12, 22, 10, 14,

In [20]:
X_all = sequence.pad_sequences(X_all, maxlen=max_sequence_size)  # to overcome the variable length issue 

In [21]:
X_all[0]   # if len(sequence < 500) -> we will pad infront of it with zeros.

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 11,  1,  1,  1,  1,  1, 14,  6,  6,  6,  6,  6,  4, 13, 15,
       15, 17,  4,  6, 19,  6, 13,  6, 19, 13,  6,  4, 19,  4, 11, 19,  9,
        6, 14, 13,  5,  3, 19,  6, 13, 15, 22, 17, 14, 10, 14, 22,  8,  6,
        4,  6,  1, 22,  6, 11, 19, 16, 16,  1, 22,  3,  7, 19, 15,  9, 17,
       15, 19,  1,  8,  9,  9,  8, 16, 13,  5,  4,  7, 14, 17, 22,  2, 14,
       15, 17, 10, 15,  4,  8, 14,  8, 10, 10, 15,  5, 15,  7,  4, 12, 19,
        8,  6,  8, 15,  3

# Now we need to split the data 

In [22]:
print(X_all.shape)  # extremely important that you view this! 
print(y_all.shape)  # make sure you are comfortable with shapes! 

(7, 500)
(7,)


We'll do a basic shuffle and 66%, 33% split. 

In [25]:
n = X_all.shape[0]  # number of data points  n=7

In [26]:
# randomize to shuffle first
randomize = np.arange(n)
np.random.shuffle(randomize)

In [27]:
randomize

array([0, 1, 4, 6, 5, 2, 3])

In [38]:
X_all = X_all[randomize] # it will randomize the data according to the sequence of randomize
y_all = y_all[randomize] # it will randomize labal according to the seqence of randomize

In [39]:
test_split = round(n * 2 / 3)
X_train = X_all[:test_split]   # start to (just before) test_split 
y_train = y_all[:test_split]   
X_test  = X_all[test_split:]   # test_split to end 
y_test  = y_all[test_split:]

In [40]:
# Print shapes again 
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5, 500)
(5,)
(2, 500)
(2,)


# Understanding the Shapes 

In [ ]:
"     [  .  .  .  .  .  5 ]         Initial shape: (5, )   "


In [ ]:
"""     [ [. . . . . . . . 500 ]    []   []   []   []  ]      

Shape is now: (5, 500)
""""""

In [ ]:
# This, when converted to one-hot representation becomes: 

""" 
[ [
    for 5 -> [0 0 0 0 0 1 0 0 0 0.....] so on to 23 
   [. . . . . . . . . . 23]             (where 23 is the number of amino acids)
    . 
    .
    .
    500 
  ]    
  []   
  []   
  []   
  []  
  ] 
  
  So, the final shape will be: (5, 500, 23)
"""

# The Model 

In [42]:
from keras.layers import Embedding, Input, Dropout, Flatten, Dense, Activation
from keras.models import Model, Sequential
from keras.optimizers import SGD

In [43]:
num_amino_acids = 23 
embedding_dims = 10  # embedding_dims embed the 23 into 10 
nb_epoch = 2
batch_size = 2      # depends on you , because our data is less so 2 is fine.

In [59]:
model = Sequential() 
model.add(Embedding(num_amino_acids, embedding_dims,input_length=max_sequence_size))

model.add(Flatten())
'''
if we didn't add the Flatten layer, it will not properly predict it , bcz final answer is in form of 
(None, 500, 1)   whereas we need it in the form of (None, 1), so predict 1, 0   
'''
model.add(Dense(25,activation='sigmoid'))
model.add(Dense(1,activation= 'sigmoid'))



In [60]:
model.summary()
'''
(None, 500, 10), where None is batch size which is up to you....in current case it's 2.
'''

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 10)           230       
_________________________________________________________________
flatten_2 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 25)                125025    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 26        
Total params: 125,281
Trainable params: 125,281
Non-trainable params: 0
_________________________________________________________________


"\n(None, 500, 10), where None is batch size which is up to you....in current case it's 2.\n"

In [61]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(),
metrics=['accuracy'])

In [62]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 10)           230       
_________________________________________________________________
flatten_2 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 25)                125025    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 26        
Total params: 125,281
Trainable params: 125,281
Non-trainable params: 0
_________________________________________________________________


In [63]:
hist = model.fit(X_train, y_train,
                  batch_size = batch_size,
                  epochs = nb_epoch, 
                  validation_data = (X_test, y_test),
                  verbose=1)   

Epoch 1/2
3/3 [==============================] - 0s 30ms/step - loss: 0.6948 - accuracy: 0.6000 - val_loss: 0.6694 - val_accuracy: 1.0000
Epoch 2/2
3/3 [==============================] - 0s 5ms/step - loss: 0.6903 - accuracy: 0.6000 - val_loss: 0.6511 - val_accuracy: 1.0000


# Changing to the Functional API 

In [65]:
'''
keras also provide the functional API for sequential model so that we will use API to train the model.

1. Each space is funtion in Functional API
2. Functional API is more powerfull that we get other information from the amino_acid which helps to train model.
'''

input = Input(shape=(max_sequence_size,)) # we give Input function to create input for model


In [66]:
embedding = Embedding(num_amino_acids,embedding_dims)(input) # pass parameter & input to Embedding fn.

In [67]:
x = Flatten()(embedding) # first () is object initiation parenthesis & #second is function call parenthesis.
x = Dense(25, activation='sigmoid')(x)
x = Dense(1)(x) # we didn't pass the activation in this layer bcz we need activation on seperate layer.

In [68]:
output = Activation('sigmoid')(x)

In [69]:
model = Model([input],output)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 500, 10)           230       
_________________________________________________________________
flatten_3 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 25)                125025    
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 26        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 125,281
Trainable params: 125,281
Non-trainable params: 0
________________________________________________

In [70]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [71]:
hist = model.fit(X_train, y_train,
                  batch_size = batch_size,
                  epochs = nb_epoch, 
                  validation_data = (X_test, y_test),
                  verbose=1)    

Epoch 1/2
3/3 [==============================] - 0s 35ms/step - loss: 0.7017 - accuracy: 0.6000 - val_loss: 0.3561 - val_accuracy: 1.0000
Epoch 2/2
3/3 [==============================] - 0s 11ms/step - loss: 0.6169 - accuracy: 0.6000 - val_loss: 0.3246 - val_accuracy: 1.0000


In [72]:
hist.history

{'loss': [0.7016654014587402, 0.6168941855430603],
 'accuracy': [0.6000000238418579, 0.6000000238418579],
 'val_loss': [0.3560737371444702, 0.3245782256126404],
 'val_accuracy': [1.0, 1.0]}